In [266]:
import pandas as pd
import numpy as np

dir = '/Users/xinwang/Downloads/'

Dot_file= dir + 'Dotloop.xlsx'
Zillow_file = dir + 'ZG.xlsx'

dot_df = pd.read_excel(Dot_file)
zillow_df = pd.read_excel(Zillow_file)

# 4   total:189
print(dot_df.shape)
# 16  total:402
print(zillow_df.shape)


(189, 18)
(402, 18)


In [295]:
dot_df.loc[dot_df['Billing Run: Number Of Invoices']==1]['Billing Run: Total Time'].median()

15554.0

In [296]:
zillow_df.loc[zillow_df['Billing Run: Number Of Invoices']==1]['Billing Run: Total Time'].median()

258.5

In [ ]:


# dot_df = dot_df[dot_df['Billing Run: Number Of Invoices']>0]
# zillow_df = zillow_df[zillow_df['Billing Run: Number Of Invoices']>0]


def process_time(df):
    df['total_times_second'] = df['Billing Run: Total Time'].apply(lambda x: x/1000) 
    
    df.rename(columns={
        'Billing Run: Billing Run Number':'billingrun_number',
        'Billing Run: Number Of Invoices':'number_of_invoice'
    }, inplace=True)
    return df

dot_df = process_time(dot_df)
zillow_df = process_time(zillow_df)

time_columns = ['billingrun_number','number_of_invoice','total_times_second']

def process_avg_invoice_time(df):
    df['avg_seconds_per_invoice'] = df['number_of_invoice'] / df['total_times_second']
    
    return df


dot_df = process_avg_invoice_time(dot_df)  
zillow_df = process_avg_invoice_time(zillow_df) 


time_columns.append('avg_seconds_per_invoice')

zillow_df_avg = zillow_df[time_columns].sort_values(
    'number_of_invoice', ascending=False)
dot_df_avg = dot_df[time_columns].sort_values(
    'number_of_invoice', ascending=False)

print(zillow_df_avg['avg_seconds_per_invoice'].iloc[:,].mean())
print(dot_df_avg['avg_seconds_per_invoice'].iloc[:,].mean())


dot_tax_time = pd.read_csv(dir +'dot_single_account_billrun_tax_times.csv')

dot_tax_time['tax_created_on']= pd.to_datetime(dot_tax_time['created_on'])
dot_tax_time['tax_updated_on']= pd.to_datetime(dot_tax_time['updated_on'])
dot_tax_time.drop(columns=['created_on','updated_on'], axis=1, inplace=True)

dot_tax_time['tax_time_seconds'] = (dot_tax_time['tax_updated_on']-dot_tax_time['tax_created_on']) / np.timedelta64(1,'s')


dot_df_avg = pd.merge(dot_df_avg,dot_tax_time, on='billingrun_number')

dot_df_avg.head()

In [265]:
dot_df_avg.to_csv(dir+'Dot_BillRun_tax_analysis.csv', index=False)